In [1]:
import pandas as pd

# Path to the .csv file in your Google Drive
file_path = '/content/drive/MyDrive/Colab Notebooks/Suicide_Detection.csv'

# Load the dataset
df = pd.read_csv(file_path)

# Check the first few rows of the dataset
print(df.head())

   Unnamed: 0                                               text        class
0           2  Ex Wife Threatening SuicideRecently I left my ...      suicide
1           3  Am I weird I don't get affected by compliments...  non-suicide
2           4  Finally 2020 is almost over... So I can never ...  non-suicide
3           8          i need helpjust help me im crying so hard      suicide
4           9  I’m so lostHello, my name is Adam (16) and I’v...      suicide


In [2]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 5.7 MB/s eta 0:00:00


In [3]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.8 MB/s eta 0:00:00


In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
import contractions
from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## 1.Removing Extra Spaces and newlines

In [5]:
def remove_blanklines(data):
  formated_text=data.replace('\\n',' ').replace('\\t',' ').replace('\\r',' ').replace('\\',' ')
  return formated_text

## 2.Replace Emojis with words.

In [6]:
!pip install emoji
import emoji
import re
def replace_emoji(data):
    return emoji.demojize(data)  # Replaces emojis with text, e.g., 😊 -> :smile:




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 10.5 MB/s eta 0:00:00


## 3.Contraction mapping

In [7]:
# def contraction_map(data):
#     expanded_text=contractions.fix(data)
#     return expanded_text
import contractions

# Safer contractions mapping
# def apply_contractions(text):
#     if isinstance(text, str) and len(text) > 1:
#         try:
#             return contractions.fix(text)
#         except Exception as e:
#             print(f"Error processing text: {text} | Error: {e}")
#             return text  # Return original text if error occurs
#     return text  # Return original text if it's empty or not a string

import contractions

def apply_contractions(text):
    if isinstance(text, str) and len(text) > 1:
        try:
            return contractions.fix(text)
        except IndexError as e:
            print(f"Skipping text due to error: {text} | Error: {e}")
            return text  # Return original text if error occurs
    return text  # Return original text if it's empty or not a string




## 4.Accented character

In [8]:
def accented_handle(data):
  fixed_text=unidecode(data)
  return fixed_text

## 5.Cleaning- Tokenization, lowercase

In [9]:
stopword_list=stopwords.words('english')
# List of words to remove from stopwords
negation_words = ["no", "not", "nor", "neither", "none", "never", "cannot", "nowhere"]

# Remove each negation word from stopwords list
for word in negation_words:
    if word in stopword_list:
        stopword_list.remove(word)
def cleaning_data(data):
  tokens=word_tokenize(data)
  clean_data=[word.lower() for word in tokens if (word.lower() not in stopword_list) and (word not in punctuation) and (word.isalpha()) and (len(word)>1)]
  return clean_data


## 6.Stemming or lemmatization

In [15]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Function to map POS tag for each word to WordNet format
def get_wordnet_pos(tag):
    """Map POS tag to first character lemmatize() accepts"""
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag[0].upper(), wordnet.NOUN)  # Default to noun if unknown

# Lemmatization function with POS tagging for each document
def lemmatizations(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = []

    # Tokenize the text into words
    tokens = word_tokenize(text)

    # POS tagging for tokens and lemmatization based on POS
    for token, pos in pos_tag(tokens):
        pos_tagged = get_wordnet_pos(pos)  # Convert POS to WordNet format
        lemmatized_word = lemmatizer.lemmatize(token.lower(), pos=pos_tagged)
        lemmatized_words.append(lemmatized_word)

    return " ".join(lemmatized_words)





In [21]:
!pip install swifter
import swifter
x_train, x_test, y_train, y_test=train_test_split( df['text'], df['class'], test_size=0.2, random_state=42, stratify= df['class'])
cleaned_train=x_train.swifter.apply(remove_blanklines)
cleaned_train=cleaned_train.swifter.apply(replace_emoji)
cleaned_train=cleaned_train.swifter.apply(apply_contractions)
cleaned_train=cleaned_train.swifter.apply(accented_handle)
cleaned_train=cleaned_train.swifter.apply(cleaning_data)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.6/242.6 kB 9.9 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=92854ec31a7eb9e5962591de77b3c465a93aeb43578fb156a6dd2aba5871e686
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


Pandas Apply:   0%|          | 0/185659 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/185659 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/185659 [00:00<?, ?it/s]

Skipping text due to error: İ am losing my mind...İ dont know how i can endure this bullshit ...  
İam 21 and suffered almost every stage of my life , things are not going on my way , worst thing is everyone hates me even my family too . They think iam a failure.
İam an university student but my grades like an  rotten apple on the tree... i have no motivation or energy. And dont have a girlfriend still virgin . Why i should keep up for nothing ?,  for more suffer ? or more failure ?
İ just want peace , love and some money...
İ know there is still some hope but i tired keep fighting it is pointless , i hate it i just want some victory . İ am looking for a gun but it is hard to access on my country . İ just dont want hurt anymore... it is enough.  İf people interested in motivational videos please watch 
(Why we choose suicide Mark Henic) it relaxed me one bit . İ need your prays too | Error: string index out of range
Skipping text due to error: İ have seen a weird dream... ( Warning ple

Pandas Apply:   0%|          | 0/185659 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/185659 [00:00<?, ?it/s]

In [22]:
# If cleaned_train contains tokenized data (lists), join the tokens back into strings
cleaned_train = cleaned_train.apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

cleaned_train=cleaned_train.swifter.apply(lemmatizations)
cleaned_train

Pandas Apply:   0%|          | 0/185659 [00:00<?, ?it/s]

,text
52852,normalise watch dub anime not care not origina...
165390,sad gorl hour school minute feel like fuckin s...
10806,every little problem make want kill useless sh...
33130,want die unhealthy reading submission make fro...
73518,reddit app icon good look weird see color
...,...
217633,post photo edit old photo dad karma whore want...
65434,take online test get question teacher need gra...
210153,think might die yeari not deal anymore not dea...
62869,determinism true point think logically decide


In [24]:
cleaned_test=x_test.swifter.apply(remove_blanklines)
cleaned_test=cleaned_test.swifter.apply(replace_emoji)
cleaned_test=cleaned_test.swifter.apply(apply_contractions)
cleaned_test=cleaned_test.swifter.apply(accented_handle)
cleaned_test=cleaned_test.swifter.apply(cleaning_data)
# Apply the same cleaning to cleaned_test (if applicable)
cleaned_test = cleaned_test.apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
cleaned_test=cleaned_test.swifter.apply(lemmatizations)
cleaned_test

Pandas Apply:   0%|          | 0/46415 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/46415 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/46415 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/46415 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/46415 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/46415 [00:00<?, ?it/s]

,text
41885,reposts karma whore matter much take something...
220532,take trazodone last nightshould worry life risk
9045,seriously think suicide way end pain experienc...
49878,think donei not continue like past two year in...
209547,lose fightit year depression up game not keep ...
...,...
76679,lady cute lady awkwardly tiny wave shuffle tac...
48903,want cut tattooed skin body want shave head sh...
59626,listen kanye album rank think
153876,hey fellow teen want say officially


# CountVectorizer

In [25]:
# # If cleaned_train contains tokenized data (lists), join the tokens back into strings
# cleaned_train = cleaned_train.apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# # Apply the same cleaning to cleaned_test (if applicable)
# cleaned_test = cleaned_test.apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Now apply CountVectorizer
count_vect = CountVectorizer(max_df=0.95, min_df=1, max_features=1000)
count_vect_train = count_vect.fit_transform(cleaned_train)
count_vect_test = count_vect.transform(cleaned_test)

# Convert to DataFrame for readability (optional)
df_train = pd.DataFrame(count_vect_train.toarray(), columns=count_vect.get_feature_names_out())
df_test = pd.DataFrame(count_vect_test.toarray(), columns=count_vect.get_feature_names_out())


In [26]:
# Display the feature names (words) created by the CountVectorizer
print(count_vect.get_feature_names_out())


['able' 'absolutely' 'abuse' 'abusive' 'accept' 'accident' 'account'
 'achieve' 'across' 'act' 'action' 'actual' 'actually' 'add' 'addict'
 'admit' 'adult' 'advice' 'affect' 'afford' 'afraid' 'age' 'ago' 'agree'
 'ahead' 'alcohol' 'alive' 'allow' 'almost' 'alone' 'along' 'alot'
 'already' 'also' 'although' 'always' 'amaze' 'among' 'amount' 'amp'
 'anger' 'angry' 'animal' 'annoy' 'another' 'answer' 'anxiety' 'anxious'
 'anybody' 'anymore' 'anyone' 'anything' 'anyway' 'anyways' 'anywhere'
 'apart' 'apartment' 'apologize' 'apparently' 'apply' 'appointment'
 'appreciate' 'area' 'argument' 'arm' 'around' 'art' 'as' 'ask' 'asleep'
 'asshole' 'attack' 'attempt' 'attention' 'attractive' 'avoid' 'award'
 'away' 'awful' 'awkward' 'baby' 'back' 'bad' 'badly' 'ball' 'barely'
 'basically' 'bathroom' 'battle' 'bear' 'beat' 'beautiful' 'become' 'bed'
 'beg' 'begin' 'behind' 'believe' 'besides' 'best' 'better' 'beyond' 'big'
 'bill' 'bipolar' 'birthday' 'bit' 'bitch' 'black' 'blame' 'block' 'blood'
 '

# TF-IDF Vectorizer

In [27]:
tfidf_vect=TfidfVectorizer(max_df=0.95, min_df=1, max_features=1000)
tfidf_vect_train=tfidf_vect.fit_transform(cleaned_train)
tfidf_vect_test=tfidf_vect.transform(cleaned_test)
pd.DataFrame(tfidf_vect_train.toarray(), columns=tfidf_vect.get_feature_names_out())

,able,absolutely,abuse,abusive,accept,accident,account,achieve,across,act,...,write,wrong,yeah,year,yell,yes,yesterday,yet,young,youtube
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.247004,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.130129,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185654,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
185655,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.181786,0.181485,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
185656,0.0,0.000000,0.162113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.084770,0.178518,0.0,0.0,0.0,0.0,0.0
185657,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [28]:
print(tfidf_vect.get_feature_names_out())

['able' 'absolutely' 'abuse' 'abusive' 'accept' 'accident' 'account'
 'achieve' 'across' 'act' 'action' 'actual' 'actually' 'add' 'addict'
 'admit' 'adult' 'advice' 'affect' 'afford' 'afraid' 'age' 'ago' 'agree'
 'ahead' 'alcohol' 'alive' 'allow' 'almost' 'alone' 'along' 'alot'
 'already' 'also' 'although' 'always' 'amaze' 'among' 'amount' 'amp'
 'anger' 'angry' 'animal' 'annoy' 'another' 'answer' 'anxiety' 'anxious'
 'anybody' 'anymore' 'anyone' 'anything' 'anyway' 'anyways' 'anywhere'
 'apart' 'apartment' 'apologize' 'apparently' 'apply' 'appointment'
 'appreciate' 'area' 'argument' 'arm' 'around' 'art' 'as' 'ask' 'asleep'
 'asshole' 'attack' 'attempt' 'attention' 'attractive' 'avoid' 'award'
 'away' 'awful' 'awkward' 'baby' 'back' 'bad' 'badly' 'ball' 'barely'
 'basically' 'bathroom' 'battle' 'bear' 'beat' 'beautiful' 'become' 'bed'
 'beg' 'begin' 'behind' 'believe' 'besides' 'best' 'better' 'beyond' 'big'
 'bill' 'bipolar' 'birthday' 'bit' 'bitch' 'black' 'blame' 'block' 'blood'
 '

In [29]:
from itertools import count
count_mnb=MultinomialNB()
count_mnb.fit(count_vect_train, y_train)
count_pred=count_mnb.predict(count_vect_test)
print(classification_report(y_test, count_pred))


              precision    recall  f1-score   support

 non-suicide       0.93      0.78      0.85     23208
     suicide       0.81      0.94      0.87     23207

    accuracy                           0.86     46415
   macro avg       0.87      0.86      0.86     46415
weighted avg       0.87      0.86      0.86     46415



In [30]:
count_accuracy=accuracy_score(y_test, count_pred)
count_accuracy


0.8615964666594851

In [31]:
tfidf_mnb=MultinomialNB()
tfidf_mnb.fit(tfidf_vect_train, y_train)
tfidf_pred=tfidf_mnb.predict(tfidf_vect_test)
print(classification_report(y_test, tfidf_pred))

              precision    recall  f1-score   support

 non-suicide       0.92      0.80      0.85     23208
     suicide       0.82      0.93      0.87     23207

    accuracy                           0.86     46415
   macro avg       0.87      0.86      0.86     46415
weighted avg       0.87      0.86      0.86     46415



In [32]:
tfidf_accuracy=accuracy_score(y_test, tfidf_pred)
tfidf_accuracy

0.863707853064742

In [33]:
!pip install gensim
from gensim.models import Word2Vec
model = Word2Vec(sentences=cleaned_train, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")


In [34]:
import numpy as np

def get_sentence_vector(sentence):
    # Get the word vectors for the words in the sentence
    word_vectors = [model.wv[word] for word in sentence if word in model.wv]
    if not word_vectors:  # If no words are found, return a zero vector
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Generate sentence vectors for your training and test data
X_train = np.array([get_sentence_vector(sentence) for sentence in cleaned_train])
X_test = np.array([get_sentence_vector(sentence) for sentence in cleaned_test])  # Assuming cleaned_test is also a list of lists


In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# Train a Random Forest classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.7863190778843047
              precision    recall  f1-score   support

 non-suicide       0.76      0.83      0.80     23208
     suicide       0.81      0.74      0.78     23207

    accuracy                           0.79     46415
   macro avg       0.79      0.79      0.79     46415
weighted avg       0.79      0.79      0.79     46415



In [36]:
import swifter
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec

# Assuming x_train and x_test are processed, and y_train and y_test are your labels

# Hyperparameter grid for RandomizedSearchCV
param_dist = {
    'C': np.logspace(-4, 4, 20),
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
}

# Dictionary to store results
results = {}

# 1. CountVectorizer
count_vect = CountVectorizer(max_df=0.95, min_df=1, max_features=1000)
count_vect_train = count_vect.fit_transform(x_train.swifter.apply(str))
count_vect_test = count_vect.transform(x_test.swifter.apply(str))

clf_count = RandomizedSearchCV(LogisticRegression(max_iter=1000), param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
clf_count.fit(count_vect_train, y_train)
y_pred_count = clf_count.predict(count_vect_test)
results['CountVectorizer'] = accuracy_score(y_test, y_pred_count)

# 2. TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_df=0.95, min_df=1, max_features=1000)
tfidf_vect_train = tfidf_vect.fit_transform(x_train.swifter.apply(str))
tfidf_vect_test = tfidf_vect.transform(x_test.swifter.apply(str))

clf_tfidf = RandomizedSearchCV(LogisticRegression(max_iter=1000), param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
clf_tfidf.fit(tfidf_vect_train, y_train)
y_pred_tfidf = clf_tfidf.predict(tfidf_vect_test)
results['TfidfVectorizer'] = accuracy_score(y_test, y_pred_tfidf)

# 3. Word2Vec
# Convert sentences into lists of words for Word2Vec
sentences_train = [sentence.split() for sentence in x_train]
sentences_test = [sentence.split() for sentence in x_test]

# Train Word2Vec model
model = Word2Vec(sentences_train, vector_size=100, window=5, min_count=1, workers=4)

# Function to generate sentence vectors
def get_sentence_vector(sentence, model):
    word_vectors = [model.wv[word] for word in sentence if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

# Generate vectors
X_train_w2v = np.array([get_sentence_vector(sentence, model) for sentence in sentences_train])
X_test_w2v = np.array([get_sentence_vector(sentence, model) for sentence in sentences_test])

clf_w2v = RandomizedSearchCV(LogisticRegression(max_iter=1000), param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
clf_w2v.fit(X_train_w2v, y_train)
y_pred_w2v = clf_w2v.predict(X_test_w2v)
results['Word2Vec'] = accuracy_score(y_test, y_pred_w2v)

# Display results
print("Accuracy Results:")
for method, accuracy in results.items():
    print(f"{method}: {accuracy:.4f}")


Pandas Apply:   0%|          | 0/185659 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/46415 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/185659 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/46415 [00:00<?, ?it/s]

Accuracy Results:
CountVectorizer: 0.9160
TfidfVectorizer: 0.9225
Word2Vec: 0.9064
